## Importing Packages

In [14]:
import Pkg

using Pkg
Pkg.add("CUDA")
Pkg.add("Flux")
Pkg.add("Transformers")

   Resolving package versions...
    Updating `C:\Users\Alper\.julia\environments\v1.6\Project.toml`
  [052768ef] + CUDA v3.5.0
  No Changes to `C:\Users\Alper\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ PyCall
  ✓ CSV
  ✓ GraphMLDatasets
  ✓ GeometricFlux
  ✓ Plots
  5 dependencies successfully precompiled in 28 seconds (210 already precompiled)
   Resolving package versions...
    Updating `C:\Users\Alper\.julia\environments\v1.6\Project.toml`
  [587475ba] + Flux v0.12.8
  No Changes to `C:\Users\Alper\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ PyCall
  ✓ CSV
  ✓ GraphMLDatasets
  ✓ GeometricFlux
  ✓ Plots
  5 dependencies successfully precompiled in 29 seconds (210 already precompiled)
   Resolving package versions...
   Installed LightXML ───────── v0.9.0
   Installed InternedStrings ── v0.7.0
   Installed BSON ───────────── v0.3.4
   Installed TupleTools ─────── v1.3.0
   Installed Strided ────────── v1.1.2
   Installed Pickle ─────

In [15]:
using Flux
using CUDA
using Transformers
using Transformers.Basic #for loading the positional embedding

┌ Info: Precompiling Transformers [21ca0261-441d-5938-ace7-c90938fde4d4]
└ @ Base loading.jl:1342


## System Info

In [50]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.dependencies()[Base.UUID("1902f260-5fb4-5aff-8c31-6271790ab950")].version)
println("GPU: ", read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String))
println("CUDA: ", CUDA.versioninfo())

OS: NT
Julia: 1.6.3
Knet: 1.4.9
GPU: Quadro RTX 3000

CUDA toolkit 11.4, artifact installation
NVIDIA driver 496.49.0, for CUDA 11.5
CUDA driver 11.5

Libraries: 
- CUBLAS: 11.5.4
- CURAND: 10.2.5
- CUFFT: 10.5.1
- CUSOLVER: 11.2.0
- CUSPARSE: 11.6.0
- CUPTI: 14.0.0
- NVML: 11.0.0+496.49
- CUDNN: 8.20.2 (for CUDA 11.4.0)


- CUTENSOR: 1.3.0 (for CUDA 11.2.0)

Toolchain:
- Julia: 1.6.3
- LLVM: 11.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80

1 device:
  0: Quadro RTX 3000 (sm_75, 4.740 GiB / 6.000 GiB available)
CUDA: nothing


# Toy test case : Copy

In [48]:
labels = collect(1:10)
startsym = 11
endsym = 12
unksym = 0
labels = [unksym, startsym, endsym, labels...]
vocab = Transformers.Basic.Vocabulary(labels, unksym)

Vocabulary{Int64}(13, unk=0)

In [24]:
#function for generate training datas
sample_data() = (d = rand(1:10, 10); (d,d))
#function for adding start & end symbol
preprocess(x) = [startsym, x..., endsym]

@show sample = preprocess.(sample_data())
@show encoded_sample = vocab(sample[1]) #use Vocabulary to encode the training data

sample = preprocess.(sample_data()) = ([11, 10, 2, 10, 3, 1, 6, 9, 2, 5, 8, 12], [11, 10, 2, 10, 3, 1, 6, 9, 2, 5, 8, 12])
encoded_sample = vocab(sample[1]) = [2, 13, 5, 13, 6, 4, 9, 12, 5, 8, 11, 3]


12-element Vector{Int64}:
  2
 13
  5
 13
  6
  4
  9
 12
  5
  8
 11
  3

# Define a model

In [25]:
#define a Word embedding layer which turn word index to word vector
embed = Embed(512, length(vocab)) |> gpu
#define a position embedding layer metioned above
pe = PositionEmbedding(512) |> gpu

#wrapper for get embedding
function embedding(x)
  we = embed(x, inv(sqrt(512))) 
  e = we .+ pe(we)
    return e
end

#define 2 layer of transformer
encode_t1 = Transformer(512, 8, 64, 2048) |> gpu  
encode_t2 = Transformer(512, 8, 64, 2048) |> gpu

#define 2 layer of transformer decoder
decode_t1 = TransformerDecoder(512, 8, 64, 2048) |> gpu  
decode_t2 = TransformerDecoder(512, 8, 64, 2048) |> gpu

#define the layer to get the final output probabilities
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

function encoder_forward(x)
  e = embedding(x)
  t1 = encode_t1(e)
  t2 = encode_t2(t1)
  return t2
end

function decoder_forward(x, m)
  e = embedding(x)
  t1 = decode_t1(e, m)
  t2 = decode_t2(t1, m)
  p = linear(t2)
    return p
end

decoder_forward (generic function with 1 method)

# Run the model on a sample

In [26]:
enc = encoder_forward(encoded_sample)
probs = decoder_forward(encoded_sample, enc)

13×12 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -2.3323   -2.34909  -2.44869  -2.49098  …  -1.99231  -1.97707  -2.10335
 -3.5336   -3.59267  -3.65139  -3.57039     -3.23912  -3.29698  -3.37359
 -1.91805  -1.99483  -2.03525  -2.1835      -2.35824  -2.26476  -2.07237
 -1.2258   -1.21493  -1.17436  -1.15963     -1.42308  -1.47211  -1.47968
 -2.99453  -3.14909  -3.33024  -3.32453     -3.74291  -3.87724  -3.8802
 -2.56449  -2.39744  -2.37409  -2.30737  …  -2.48594  -2.2813   -2.10433
 -4.86829  -4.82257  -4.98728  -5.13294     -5.1413   -5.21062  -5.302
 -3.9087   -3.85332  -3.71741  -3.74136     -4.03006  -3.96037  -3.77018
 -2.48812  -2.55806  -2.66239  -2.76245     -2.23914  -2.41032  -2.62349
 -2.1162   -2.04534  -1.98752  -1.91218     -1.89573  -1.93287  -2.00474
 -2.80382  -2.83438  -2.7462   -2.65263  …  -2.57856  -2.48288  -2.44479
 -5.08103  -5.11392  -5.0582   -5.03105     -4.27402  -4.27918  -4.27636
 -4.80408  -4.64663  -4.54454  -4.63024     -4.44452  -4.42397  -4.4566

# Define the loss and train loop

In [91]:
# TODO: find the problem with the onehot func.

function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

#define loss function
function loss(x, y)
  label = Flux.onehot(vocab, y) #turn the index to one-hot encoding
  label = smooth(label) #perform label smoothing
  enc = encoder_forward(x)
	probs = decoder_forward(y, enc)
  l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])
  return l
end

#collect all the parameters
ps = params(embed, pe, encode_t1, encode_t2, decode_t1, decode_t2, linear)
opt = ADAM(1e-4)

#function for created batched data
using Transformers.Datasets: batched

#flux function for update parameters
using Flux: gradient
using Flux.Optimise: update!

#define training loop
function train!()
  @info "start training"
  for i = 1:2000
    data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
		x, y = preprocess.(data[1]), preprocess.(data[2])
    x, y = vocab(x), vocab(y)#encode the data
    x, y = todevice(x, y) #move to gpu
    l = loss(x, y)
    grad = gradient(()->l, ps)
    if i % 8 == 0
    	println("loss = $l")
    end
    update!(opt, ps, grad)
  end
end

train! (generic function with 1 method)

In [92]:
train!()

┌ Info: start training
└ @ Main In[91]:31


LoadError: MethodError: no method matching OneHotArray(::Transformers.Basic.OneHot{0x0000000d})
[0mClosest candidates are:
[0m  OneHotArray(::Any, [91m::Transformers.Basic.OneHot[39m) at C:\Users\Alper\.julia\packages\Transformers\3YgSd\src\basic\embeds\onehot.jl:115
[0m  OneHotArray(::Any, [91m::Any[39m) at C:\Users\Alper\.julia\packages\Transformers\3YgSd\src\basic\embeds\onehot.jl:114
[0m  OneHotArray([91m::A[39m) where {K, A<:(AbstractArray{Transformers.Basic.OneHot{K}, N} where N)} at C:\Users\Alper\.julia\packages\Transformers\3YgSd\src\basic\embeds\onehot.jl:112